# Исследование надежности заемщиков


Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


## Откройте таблицу и изучите общую информацию о данных

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`.

In [6]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть, тем более этот столбец не понадобится нам для исследования.

Выведем перечень уникальных значений столбца `children`.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения.

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски заполнены.

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру.

In [14]:
data['education'] = data['education'].str.lower()

Выведем количество строк-дубликатов в данных и удалим такие, если они есть.

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

На основании диапазонов, создаим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. 

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведем перечень уникальных целей взятия кредита из столбца `purpose`.

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создаим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.


In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуйте данные и ответьте на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [22]:
data_children = data.pivot_table(index= 'children', columns= 'debt', values= 'family_status', aggfunc= 'count')
print(len(data[data['debt'].isna()])) 
data_children = data_children.fillna(0)
data_children['all'] = data_children[1] + data_children[0]
data_children['percent'] = data_children[0] / data_children['all']
children = data_children.sort_values('percent')
print(children)

0
debt            0       1      all   percent
children                                    
4            37.0     4.0     41.0  0.902439
2          1858.0   194.0   2052.0  0.905458
1          4364.0   444.0   4808.0  0.907654
3           303.0    27.0    330.0  0.918182
0         13028.0  1063.0  14091.0  0.924562
5             9.0     0.0      9.0  1.000000


**Вывод:** Зависимость между количеством детей и возвратом кредита в срок на основании данных есть - чем меньше детей, тем выше вероятность возврата в срок, при этом заемщики с 3 детьми платят в срок чаще, чем заемщики с 1 или 2. Тем не менее слишком уж близки доли, чуть измениться выборка и значения могут измениться в другую сторону. То есть можно сделать вывод, что сколь угодно значимой разницы между группами нет.

Еще можно увидеть в наших данных, то, что по некоторым группам заемщиков у нас недостаточно данных. Выборка не сбалансирована, классы в выборке имеют разные размеры.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
data_family = data.pivot_table(index= 'family_status', columns= 'debt', values= 'education_id', aggfunc= 'count')
data_family['all'] = data_family[1] + data_family[0]
data_family['percent'] = data_family[1] / data_family['all']
family = data_family.sort_values('percent')
print(family)

debt                       0    1    all   percent
family_status                                     
вдовец / вдова           888   63    951  0.066246
в разводе               1105   84   1189  0.070648
женат / замужем        11334  927  12261  0.075606
гражданский брак        3749  385   4134  0.093130
Не женат / не замужем   2523  273   2796  0.097639


**Вывод:** Зависимость есть - заемщики, никогда не состоявшие в браке, возвращают кредит в срок реже, а состоящие в браке имеют больший процент возврата в срок, чем овдовевшие и разведенные. Кредитному отделу следует с осторожностью относиться к клиентам, которые не состоят или не состояли в браке. Согласно данным такие клиенты менее ответственные. Получается, что узаконенные отношения (включая и бывшие отношения) ведут к более ответственному отношению к своим обязательствам. Ну вроде как совпадает со здравым смыслом - семейные люди более скрупулёзно берегут свой семейное спокойствие и несут за него ответственность.

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [24]:
data_income = data.pivot_table(index= 'total_income_category', columns= 'debt', values= 'education_id', aggfunc= 'count')
data_income['all'] = data_income[1] + data_income[0]
data_income['percent'] = data_income[1] / data_income['all']
inc = data_income.sort_values('percent')
print(inc)

debt                       0     1    all   percent
total_income_category                              
D                        328    21    349  0.060172
B                       4660   354   5014  0.070602
A                         23     2     25  0.080000
C                      14568  1353  15921  0.084982
E                         20     2     22  0.090909


**Вывод:** Ожидаемо, для группы с низким доходом (Е) процент невозвратов самый высокий. Что более интересно, так это то, что с ростом доходов тенденция (D->C и B->А) к снижению числа долгов не сохраняется, а скорее даже растёт. Можно сказать, что по какой-то причине люди "перестают считать деньги" или планировать "наперёд". Из-за подобного поведения наших групп параметр ("доход") нельзя назвать надёжным при определении платёжеспособности клиента. Без дополнительной информации он может даже наоборот привести к выдаче кредита состоятельному, но не слишком "надёжному" клиенту, хотя, возможно, это может быть аномалия, вызванная недостаточным размером выборки для клиентов с категорией доходов А. Однако 2174 пропуска в данных столбца "total_income" ставят под сомнение полученный нами результат

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [25]:
data_purpose = data.pivot_table(index= 'purpose_category', columns= 'debt', values= 'education_id', aggfunc= 'count')
data_purpose['all'] = data_purpose[1] + data_purpose[0]
data_purpose['percent'] = data_purpose[1] / data_purpose['all']
purp = data_purpose.sort_values('percent')
print(purp)

debt                         0    1    all   percent
purpose_category                                    
операции с недвижимостью  9971  780  10751  0.072551
проведение свадьбы        2130  183   2313  0.079118
получение образования     3619  369   3988  0.092528
операции с автомобилем    3879  400   4279  0.093480


**Вывод:** Есть завсимость между целью кредита и возвратом в срок - заемщики, которые берут деньги на автомобиль или получение образования попадают в просрочку чаще, чем те, кому кредит нужен для покупки недвижимости или проведения свадьбы. 

#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

*Ответ:* В наших данных пропуски в столбцах "трудовой стаж" и "ежемесячный доход" абсолютно идентичны. Количество строк с отсутствующими данными в обоих указанных столбцах совпадает и равно 2174. Маловероятно, что специалисты по выдаче кредитов, заполнив все без исключения ячейки других столбцов по каждому кредитополучателю, оставляли незаполненными именно два показателя, и сделали это ровно 2174 раза в каждом столбце. Даже не проводя проверку, во всех ли случаях незаполненные ячейки находятся в одной и то же строке с высокой долей вероятности можно утверждать, что произошла техническая ошибка при формировании таблицы с исходными данными (либо некий сбой во время выгрузки данных на анализ, либо при объединении данных в единый файл, который также должен был передаваться на анализ, либо эти данные не собирались)

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* Пропуски в количественных переменных лучше заполнить медианным значением, если в данных наблюдаются "выбросы", то даже небольшое их количество может сильно "перетягивать" среднее значение в большую или меньшую сторону. Медиана зачастую даёт более правдивую картину.

### Шаг 4: общий вывод.

Было проведено исследование, заказчиком которого был кредитный отдел банка. Нужно было разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов. Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Перед проведением исследования были выдвинуты несколько гипотез:
1) Количество детей влияет на факт погашения кредита в срок
2) Семейное положение влияет на факт погашения кредита в срок
3) Цель кредита влияет на факт погашения кредита в срок
4) Уровень дохода влияет на факт погашения кредита в срок

В результате исследования было выявлено:
1) Зависимость между количеством детей и возвратом кредита в срок на основании данных есть - чем меньше детей, тем выше вероятность возврата в срок, при этом заемщики с 3 детьми платят в срок чаще, чем заемщики с 1 или 2. Тем не менее слишком уж близки доли, а так же по некоторым группам заемщиков у нас недостаточно данных. Выборка не сбалансирована, а значит и подвтердить или опровергнуть гипотезу наверняка нельзя.
2) Заемщики, никогда не состоявшие в браке, возвращают кредит в срок реже, а состоящие в браке имеют больший процент возврата в срок, чем овдовевшие и разведенные. Кредитному отделу следует с осторожностью относиться к клиентам, которые не состоят или не состояли в браке, согласно данным такие клиенты менее ответственные. Получается, что узаконенные отношения (включая и бывшие отношения) ведут к более ответственному отношению к своим обязательствам.
3) Есть завсимость между целью кредита и возвратом в срок - заемщики, которые берут деньги на автомобиль или получение образования попадают в просрочку чаще, чем те, кому кредит нужен для покупки недвижимости или проведения свадьбы. 
4) Нельзя с четкостью сказать, что уровень дохода 100% влияет на факт возврата кредита в срок. Ожидаемо, для группы с низким доходом (Е) процент невозвратов самый высокий. Что более интересно, так это то, что с ростом доходов тенденция к снижению числа долгов не сохраняется, а скорее даже растёт. Из-за подобного поведения наших групп параметр ("доход") нельзя назвать надёжным при определении платёжеспособности клиента. 

Кредитным специалистом следует брать во внимание выявленные нами закономерности между возвратом кредита в срок и целью кредита и семейным положением. 
Для более полного исследования закономерностей выборка должна быть более равномерной.
